# Libraries

In [1]:
import jupyternotify
import requests
import pandas as pd
import calendar
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from datetime import datetime
from datetime import timedelta
from time import sleep
import json
import os

ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
pd.options.display.float_format = '{:,.2f}'.format

<IPython.core.display.Javascript object>

# API Params

In [2]:
header_sales = {
    'Content-Type': 'application/json',
    'Authorization': 'ISVToken 682a99604c5a4cb4ab4cf222090ef79e'
}

In [3]:
url_sales = 'https://api.instoreview.cl/api/v2/download-zone/sales/'

In [4]:
body_sales = {
    'view_type':'week',
    'dates': [],
    'views': ['Unidades', 'Precio Lista'],
    'hierarchy':{
        'Cadena': [],
        'Sub Cadena': [],
        'Suc. ID': [],
        'EAN': [],
        'Descripción Producto': [],
        'Pro.Pst.ID': [],
        'Cad. ID':[]
    }
}

In [5]:
def clean_numbers(num):
    try:
        num + 0
    except:
        num = float(str(num).replace('.','').replace(',',''))/1000000
    return num

In [6]:
def unpack_file(response):
    try:
        file = urlopen(response.json()['download_url'])
        zip_file = ZipFile(BytesIO(file.read()))
        df = pd.read_csv(zip_file.open(zip_file.namelist()[0]), encoding='latin-1', sep=',')
    except:
        df = pd.DataFrame()
    return df

In [7]:
def check_status(response, date):
    status = response.status_code
    if status != 200:
        print("Problems in '{}':".format(date), '\n', 'Error {0}: {1}'.format(status, resp_sales.text))

In [8]:
def download_sales(year, num_week, url, header, body):
    body["dates"] = ['{0}-W{1}'.format(year, str(num_week).zfill(2))]
    resp_sales = requests.post(url, data=json.dumps(body), headers=header)
    check_status(resp_sales, body_sales["dates"][0])
    df = unpack_file(resp_sales)
    return df

In [9]:
def clean_views(data, views):
    for col in views:
        data[col] = data[col].map(clean_numbers).copy()
    return data

In [10]:
def clean_hierarchy(data, hierarchy):
    import numpy as np
    for col in hierarchy:
        data.loc[data[col]=='No Definido', col] = np.nan
    data['EAN'] = data['EAN'].astype(str)
    return data

In [11]:
def clean_sales(df, views, hierarchy):
    data = df.copy()
    df_views = clean_views(data[views], views)
    df_hierarchy = clean_hierarchy(data[hierarchy], hierarchy)
    return pd.concat([df_hierarchy, df_views], axis=1)

In [12]:
ean_to_check = [
    650240020803, 
    650240024306, 
    650240026591, 
    650240026607, 
    650240026614, 
    7501199407036, 
    74761142, 
    650240050398, 
    7501073101227, 
    650240053931, 
    650240053955, 
    650240053962, 
    650240053979, 
    650240053993, 
    650240054020, 
    650240054037, 
    650240054068
]

Solamente del cliente **Soriana**:

In [13]:
body_sales['hierarchy']['EAN'] = ean_to_check

body_sales['hierarchy']['Cadena'] = ['Soriana']

In [16]:
year_week = [(2022, w) for w in range(1, 9)]
year_week += [(2021, 52)]

In [20]:
%%time
df = pd.DataFrame()
for year, week in year_week:
    aux = download_sales(year, week, url=url_sales, body=body_sales, header=header_sales)
    df = pd.concat([df, aux], axis=0)
    sleep(20)

Wall time: 44.7 s


In [21]:
df.shape

(615, 10)

In [22]:
df.dtypes

Semanas                  object
Cadena                   object
Sub Cadena               object
Suc. ID                   int64
Cad. ID                   int64
EAN                       int64
Descripción Producto     object
Pro.Pst.ID                int64
Unidades                float64
Precio Lista             object
dtype: object

In [23]:
df[:3]

,Semanas,Cadena,Sub Cadena,Suc. ID,Cad. ID,EAN,Descripción Producto,Pro.Pst.ID,Unidades,Precio Lista
0,Semana 01 03-01-2022 a 09-01-2022,Soriana,Soriana,1187,15,650240050398,Alliviax 20 Tabletas 550mg,8556,1.00,113.240000
1,Semana 01 03-01-2022 a 09-01-2022,Soriana,Soriana,1044,15,650240050398,Alliviax 20 Tabletas 550mg,8556,1.00,113.240000
2,Semana 01 03-01-2022 a 09-01-2022,Soriana,Soriana,73290,15,650240050398,Alliviax 20 Tabletas 550mg,8556,2.00,226.480000
